In [32]:
! pip install transformers

In [14]:
from transformers import AutoTokenizer, TapexTokenizer, BartForConditionalGeneration

In [15]:
import pandas as pd

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
sales_Data = pd.read_csv("/content/drive/MyDrive/10000 Sales Records.csv")

In [18]:
sales_Data.head()

,Region,Country,Item Type,Sales Channel,Order Priority,Order Date,Order ID,Ship Date,Units Sold,Unit Price,Unit Cost,Total Revenue,Total Cost,Total Profit
0,Sub-Saharan Africa,Chad,Office Supplies,Online,L,1/27/2011,292494523,2/12/2011,4484,651.21,524.96,2920025.64,2353920.64,566105.00
1,Europe,Latvia,Beverages,Online,C,12/28/2015,361825549,1/23/2016,1075,47.45,31.79,51008.75,34174.25,16834.50
2,Middle East and North Africa,Pakistan,Vegetables,Offline,C,1/13/2011,141515767,2/1/2011,6515,154.06,90.93,1003700.90,592408.95,411291.95
3,Sub-Saharan Africa,Democratic Republic of the Congo,Household,Online,C,9/11/2012,500364005,10/6/2012,7683,668.27,502.54,5134318.41,3861014.82,1273303.59
4,Europe,Czech Republic,Beverages,Online,C,10/27/2015,127481591,12/5/2015,3491,47.45,31.79,165647.95,110978.89,54669.06


In [19]:
sales_Data = sales_Data.astype(str)

In [23]:
tokenizer = TapexTokenizer.from_pretrained("microsoft/tapex-large-finetuned-wtq")
model = BartForConditionalGeneration.from_pretrained("microsoft/tapex-large-finetuned-wtq")

In [24]:
# Specify your query
query = "What percentage of total sales come from Online orders?"

# Truncate the input to fit within the model's maximum sequence length
max_length = model.config.max_position_embeddings
encoding = tokenizer(table=sales_Data, query=query, return_tensors="pt", truncation=True, max_length=max_length)

# Generate the output
outputs = model.generate(**encoding)

# Decode the output
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

[' 50.00%']


In [25]:
#model with loaded tokenizer
tokenizer = TapexTokenizer.from_pretrained("microsoft/tapex-large-finetuned-wtq")
model = BartForConditionalGeneration.from_pretrained("microsoft/tapex-large-finetuned-wtq")

In [26]:
sales_Data = sales_Data.astype(str)

In [28]:
def predict_with_model(model, tokenizer, table, query):
    # Tokenize the query and table
    max_length = model.config.max_position_embeddings
    encoding = tokenizer(table=table, query=query, return_tensors="pt", truncation=True, max_length=max_length)

    # Generate the output
    outputs = model.generate(**encoding)

    # Decode the output
    prediction = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    return prediction



In [30]:
while True:
    query = input("Enter a query (or type 'exit' to stop): ")
    if query.lower() == 'exit':
        break

    # Predict using the model and table
    prediction = predict_with_model(model, tokenizer,sales_Data , query)

    # Print the prediction
    print(f"Query: {query}")
    print(f"Answer: {prediction}")
    print()

Enter a query (or type 'exit' to stop): What is the most profitable item (considering both unit price and unit cost)?
Query: What is the most profitable item (considering both unit price and unit cost)?
Answer:  office supplies

Enter a query (or type 'exit' to stop): Which item type has the highest total cost?
Query: Which item type has the highest total cost?
Answer:  office supplies

Enter a query (or type 'exit' to stop): What is the average order value for each item type (Office Supplies, Beverages, etc.)?
Query: What is the average order value for each item type (Office Supplies, Beverages, etc.)?
Answer:  115567.75

Enter a query (or type 'exit' to stop): exit
